View 1: Monthly Performance | Visualização 1: Desempenho Mensal

In [ ]:
monthly = clean_data.groupby(pd.Grouper(key='Month, Year', freq='M')).agg(
    New_Players=('Player Game Count', 'sum'),
    Total_GGR=('GGR Ucur', 'sum')
)

fig, ax1 = plt.subplots(figsize=(14,7))
ax2 = ax1.twinx()

ax1.plot(monthly.index, monthly['New_Players], 'g-', label='Novos Jogadores')
ax2.plot(monthly.index, monthly['Total_GGR'], 'b-', label='GGR Total')

View 2: Quarterly Variation | Visualização 2: Variação Trimestral

In [ ]:
quarterly = clean_data.groupby('Quarter').agg(
    GGR_Sum=('GGR Ucur', 'sum'),
    Wager_Sum=('Wager Ucur', 'sum')
).pct_change() * 100

plt.figure(figsize=(10,6))
quarterly['GGR_Sum'].plot(kind='bar', color=quarterly['GGR_Sum'].apply(
    lambda x: 'green' if x > 0 else 'red'))

View 3: Blackjack Analysis | Visualização 3: Anpalise de Blackjack

In [ ]:
blackjack = clean_data[clean_data['Game category'] == 'Blackjack']
morning = blackjack[blackjack['Hour'] < 12]
evening = blackjack[blackjack['Hour'] >= 12]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,7))
morning['Table Type Commercial'].value_counts().plot.pie(ax=ax1, title='Manhã')
evening['Table Type Commercial'].value_counts().plot.pie(ax=ax2, title='Tarde')

View 4: Ranking by Country | Visualização 4: Ranking por País

In [ ]:
country_rank = clean_data.groupby(['Country', 'Game category'])['GGR Ucur'].sum().unstack()
country_rank['Total'] = country_rank.sum(axis=1)
country_rank = country_rank.sort_values('Total', ascending=False)
country_rank.head(5).plot(kind='bar', stacked=True, figsize=(14,7))